# Environment Set Up

In [2]:
import os
from dotenv import load_dotenv

# Loading environment variables from .env
load_dotenv()

# Changing directory to main directory for easy data access
working_directory = os.getenv("WORKING_DIRECTORY")
os.chdir(working_directory)

# Checking the change
%pwd

'/workspaces/TumorTracer'

In [3]:
from pathlib import Path

# Checking the change
print("Git folder exists:", Path(".git").exists())

Git folder exists: True


# 4. Serve Best Model

In [ ]:
from dataclasses import dataclass
from pathlib import Path
from cnnClassifier import get_logger
from typing import Optional, Dict, Any

# Initializing the logger
logger = get_logger()

@dataclass(frozen=True)
class ModelSelectorConfig:
	"""
	Immutable configuration class to store all parameters 
	and paths required for selecting the best model. 
	"""
	source_dir: Path			# Directory with all the models
	destination_dir: Path		# Directory to store best model and its class_indices
	model_path: Path			# Path to save final model
	indices_path: Path			# Path save indices related to the final model
	filename_regex: str			# Regex pattern to extract relevent grading values

In [5]:
from cnnClassifier.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from cnnClassifier.utils.common import read_yaml, create_directories
from cnnClassifier import get_logger

# Initializing the logger
logger = get_logger()

class ConfigurationManager:
    def __init__(self, config_file_path=CONFIG_FILE_PATH, params_file_path=PARAMS_FILE_PATH) -> None:
        """
        Reads configuration files (config.yaml and params.yaml), 
        ensures necessary directories exist, and prepares structured config objects.

        Args:
        - config_file_path (str): Path to the config.yaml file.
        - params_file_path (str): Path to the params.yaml file.
        """
        # Validate and load config.yaml
        if not Path(config_file_path).exists():
            logger.error(f"Config file not found at: {config_file_path}")
            raise FileNotFoundError(f"Config file not found at: {config_file_path}")
        self.config = read_yaml(config_file_path)

        # Validate and load params.yaml
        if not Path(config_file_path).exists():
            logger.error(f"Params file not found at: {params_file_path}")
            raise FileNotFoundError(f"Params file not found at: {params_file_path}")
        self.params = read_yaml(params_file_path)

        logger.info(f"Loading configuration from {config_file_path} and parameters from {params_file_path}")

        # Create the root artifacts directory (if not already present)
        create_directories([self.config.artifacts_root])


    def get_model_selector_config(self) -> ModelSelectorConfig:
        """
        Prepares and returns the ModelSelectorConfig object.

        Returns:
        - ModelSelectorConfig: Structured config for getting the best model
        """
        config = self.config.model_selector
        params = self.params.model_selector

        model_selector_config = ModelSelectorConfig(
			source_dir=Path(config.source_dir),
			destination_dir=Path(config.destination_dir),
            model_path=Path(config.model_path),
            indices_path=Path(config.indices_path),
			filename_regex=params.FILENAME_REGEX,
        )

        logger.info(f"ModelSelectorConfig created with: {model_selector_config}")

        return model_selector_config

In [ ]:
import re
import shutil

from pathlib import Path

from cnnClassifier.utils.common import create_directories
from cnnClassifier import get_logger

# Initializing the logger
logger = get_logger()

class ModelSelection:
	def __init__(self, config: ModelSelectorConfig) -> None:
		"""
		Initializes the ModelSelection class with configuration.

		Validates the regex pattern used for parsing model filenames.
        """
		self.config = config
		self.best_model_path = None
		self.class_indices_path = None
		self.best_vacc = -1.0

		self._validate_regex_pattern()
		self.select_model()
		self.store_best_model()
        

	def _validate_regex_pattern(self) -> None:
		"""
		Validates the regex pattern with a test filename.
		"""
		try:
			test_filename = "model_e02_acc0.6297_vacc0.7083.keras"
			regex_match = re.match(self.config.filename_regex, test_filename)

			assert regex_match is not None, f"Regex failed to match filename: {test_filename}"
			assert regex_match.group(1) == "0.7083", f"Expected '0.7083', got '{regex_match.group(1)}'"

			logger.info(f"Regex loaded and validated successfully.")

		except AssertionError as exception_error:
			logger.error(f"Regex validation failed: {exception_error}")

		except re.error as exception_error:
			logger.error(f"Invalid regex pattern provided: {exception_error}")

		except Exception as exception_error:
			logger.error(f"Unexpected error during regex validation: {exception_error}")


	def select_model(self) -> None:
		"""
		Iterates over checkpoint folders to find the model with highest validation accuracy (vacc).
		"""
		source_directory = Path(self.config.source_dir)

		if not source_directory.exists():
			logger.error(f"Source directory does not exist at: {source_directory}")
			return

		try:
			# Looping all Checkpoint_* folders in soruce-directory
			for subfolder in source_directory.glob("Checkpoint_*"):
				if not subfolder.is_dir():
					continue
			
				# Search for model files
				for model_file in subfolder.glob("model_*.keras"):
					regex_match = re.match(self.config.filename_regex, model_file.name)

					if not regex_match:
						continue
						
					try:
						vacc = float(regex_match.group(1))
					except (IndexError, ValueError):
						logger.warning(f"Skipping file (invalid vacc group): {model_file.name}")
						continue
					
					# Check for best validation accuracy
					if vacc > self.best_vacc:
						self.best_model_path = model_file
						self.class_indices_path = subfolder / "class_indices.json"
						self.best_vacc = vacc

			if not self.best_model_path or not self.class_indices_path:
				logger.error(f"No suitable model found during selection at: {source_directory}")

		except Exception as exception_error:
			logger.error(f"Unexpected error during model selection: {exception_error}")


	def store_best_model(self) -> None:
		"""
		 Stores the selected best model and its class indices file into the destination directory.
		"""
		if not self.best_model_path or not self.class_indices_path:
			logger.error(f"Unable to find model or indices. Please run select_model() before store_best_model().")
			return

		try:
			# Create directory to store the models
			destination_directory = Path(self.config.destination_dir)
			create_directories([destination_directory])

			# Copy best model and class indices file to final location
			shutil.copy(self.best_model_path, self.config.model_path)
			shutil.copy(self.class_indices_path, self.config.indices_path)

			logger.info(f"Best model (vacc={self.best_vacc:.4f}) copied to: {destination_directory}")

		except Exception as exception_error:
			logger.error(f"Unexpected error during saving best model: {exception_error}")

In [ ]:
try:
	config_manager = ConfigurationManager()
	selector_config = config_manager.get_model_selector_config()

	ModelSelection(config=selector_config)

except Exception as exception:
	logger.exception(f"Unexpected error during model selection pipeline: {exception}")
	raise exception

[2025-07-11 12:39:19,377: INFO: common: YAML file: config/config.yaml loaded successfully]
[2025-07-11 12:39:19,383: INFO: common: YAML file: params/params.yaml loaded successfully]
[2025-07-11 12:39:19,384: INFO: 4188591379: Loading configuration from config/config.yaml and parameters from params/params.yaml]
[2025-07-11 12:39:19,386: INFO: common: Directory: artifacts created successfully.]
[2025-07-11 12:39:19,387: INFO: 4188591379: ModelSelectorConfig created with: ModelSelectorConfig(source_dir=PosixPath('artifacts/model_training'), destination_dir=PosixPath('model'), model_path=PosixPath('model/trained_model.keras'), indices_path=PosixPath('model/class_indices.json'), filename_regex='model_e\\d+_acc[\\d.]+_vacc([\\d.]+)\\.keras')]
[2025-07-11 12:39:19,387: INFO: 1532966547: Regex loaded and validated successfully.]
[2025-07-11 12:39:19,389: INFO: common: Directory: model created successfully.]
[2025-07-11 12:39:19,501: INFO: 1532966547: Best model (vacc=0.7083) copied to: model]
